# Create numpy grid for training for example CNNs

In [201]:
import geopandas as gpd
import pandas as pd
import shapely
import numpy as np
import pickle

In [211]:
# load cbs_od_koppel file
with open('cbs_id_koppel.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [216]:
br = pd.read_csv("../Data/filled/2015/BRfilled_data15.csv", index_col = 0)
br.geometry = br.geometry.apply(shapely.wkt.loads)
br = br.set_geometry("geometry")

In [217]:
br.C28992R100 = br.C28992R100.map(b)

In [219]:
br = br.drop(["BU_CODE"], axis = 1)

In [250]:
def to_np_grid(gpd, grid_size = 100):
    """
    args: GeoDataFrame with geometry collumn
    """
    minx = gpd.geometry.total_bounds[0]
    miny = gpd.geometry.total_bounds[1]
    
    # translate geometry so that left lower corner is 0,0
    gpd["geometry"] = gpd["geometry"].translate(-minx, -miny)
    
    # Get x,y,z shape of the df for the numpy zeros grid
    x = int((zero_grid.total_bounds / 100)[2])
    y = int((zero_grid.total_bounds / 100)[3])
    z = br.shape[1] -1 # n collumns (- geometry column)
    grid = np.zeros((x,y,z))
    
    # loop over items in gpd dataframe and add data to the numpy grid
    for _, i in gpd.iterrows(): 
        grid[int(i.geometry.bounds[0]/100), int(i.geometry.bounds[1]/100)] = i.drop("geometry")
        
    return grid

In [252]:
grid = to_np_grid(br)

# #############################################

create cbs C28992R100 to id file

In [232]:
vierkant = gpd.read_file("../Data/NLvierkant/NL_vierkant100m.shp")

KeyboardInterrupt: 

In [174]:
vierkant.reset_index().rename(columns = {"index" : "id"}).id.values

array([      0,       1,       2, ..., 3723285, 3723286, 3723287],
      dtype=int64)

In [193]:
zip_iterator = zip(vierkant.reset_index().rename(columns = {"index" : "id"}).C28992R100.values, vierkant.reset_index().rename(columns = {"index" : "id"}).id.values)

In [194]:
cbs_vierkant_dict = dict(zip_iterator)

In [200]:
br.C28992R100 = br.C28992R100.map(cbs_vierkant_dict)

In [198]:
br.C28992R100.iloc[0]

'E1824N3269'

In [204]:
with open('cbs_id_koppel.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(cbs_vierkant_dict, f, pickle.HIGHEST_PROTOCOL)